# Setting up Paths

In [4]:
#path to the data
pathCOMPASOutput = '/Volumes/Virgo/DATA/BHNS/Fiducial/'
#path to compasrepo which contains ther popsynth folder
pathRepo = '/Users/floorbroekgaarden/Projects/BHNS_project/'


#path to Bootstrap
# pathBootstrap = '/home/cneijssel/Documents/Projects/Data/CosmicInt/'


# Imports

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import h5py as h5
import time
import os
import sys
import copy
#Quick fudge to make import from ../Scripts work
import sys
sys.path.append(pathRepo + 'Scripts')


# Custom Imports from COMPASrepo

In [7]:
import ClassCosmicIntegrator  as CI #Given settings and redshifts returns rates (2D arrays) Loads the data
import ClassBootStrap         as CB 
import coencodeVarious        as CV



# General Idea

In the previous notebook we cerated an h5 file with the rate per chirp mass per system per variation. In this notebook we will make a table of the rates. In addition we want to also bootstrap the rates. 

In [4]:
h5file           = h5.File('VariationIntegration.h5')

# Results        a 2D array of 18 variations 2X3X3 (rows) and 
#                              6 types of rates 2X3 (columns) and + - error so 18 columns
results          = np.zeros(shape=(18,18))

#keys are not given sorted, however in the sorted order it is easier to make the table
#convert list of string numbers to integers, get the arguments to sort and use these
#to sort keys
a = np.array(h5file.keys())
b = np.argsort(a.astype(int))

for v in a[b]:
    #print h5file[v]['SFR_MZ_GSMF'][...]
    print h5file[v]['BBH'].keys()

[u'IntrinsicRedshift0', u'ObservedRate', u'Pessimistic', u'chirpMass', u'randomSeed']
[u'IntrinsicRedshift0', u'ObservedRate', u'Pessimistic', u'chirpMass', u'randomSeed']
[u'IntrinsicRedshift0', u'ObservedRate', u'Pessimistic', u'chirpMass', u'randomSeed']
[u'IntrinsicRedshift0', u'ObservedRate', u'Pessimistic', u'chirpMass', u'randomSeed']
[u'IntrinsicRedshift0', u'ObservedRate', u'Pessimistic', u'chirpMass', u'randomSeed']
[u'IntrinsicRedshift0', u'ObservedRate', u'Pessimistic', u'chirpMass', u'randomSeed']
[u'IntrinsicRedshift0', u'ObservedRate', u'Pessimistic', u'chirpMass', u'randomSeed']
[u'IntrinsicRedshift0', u'ObservedRate', u'Pessimistic', u'chirpMass', u'randomSeed']
[u'IntrinsicRedshift0', u'ObservedRate', u'Pessimistic', u'chirpMass', u'randomSeed']
[u'IntrinsicRedshift0', u'ObservedRate', u'Pessimistic', u'chirpMass', u'randomSeed']
[u'IntrinsicRedshift0', u'ObservedRate', u'Pessimistic', u'chirpMass', u'randomSeed']
[u'IntrinsicRedshift0', u'ObservedRate', u'Pessimistic

# Bootstrap

I have premade a bootsStrap wit ClassBootstrap, this is a resampling off all seeds per metallicity with replacement. From that I count of *all* the DCOs in my double compact object file how often they appear. 
That is stored as a column to file. Hence if I have a maskDCO of one file say cosmic integration, that same maskDCO applied to the bootstrap file gives me the nr of draws. Hence here I only have to recover the mask, and apply to bootstrap file to get counts. Then I sort the counts and get +- 90% uncertainty

In [5]:
boot   = CB.Bootstrap(pathToFile = pathBootstrap, fileName='Bootstrap5.h5')

def bootstrap(ratePerSystem):
    #our bootstrap has 200 samples 
    nrSamples = 200
    #what will be the total rate for each bootstrap
    Rates     = np.zeros(nrSamples)
    for sample in range(nrSamples):
        #Reset rates according to Bootstrap sample
        s             = boot.ReadSample(nrSample=sample, maskDCO=maskDCO)
        Rates[sample] = np.sum(np.multiply(s, ratePerSystem))
        
    #order them from low to high and give error 
    Rates     = np.sort(Rates) - np.sum(ratePerSystem)
    #If you sort array at what index do you find lower and upper confidence 90%
    lower     = int((nrSamples/100.)*5)
    upper     = int((nrSamples/100.)*95)
    return Rates[lower], Rates[upper]
    

In [6]:
#per sorted variation (row)
redoCalculation = False
if redoCalculation:
    for Pessimistic in [False, True]:
        counterRow = 0
        for v in a[b]:
            print h5file[v]['SFR_MZ_GSMF'][...]
            #per type (column)
            counterColumn   = 0
            types           = ['BBH', 'BHNS', 'BNS']
            originalDCOfile = h5.File(pathCOMPASOutput+'COMPASOutput.h5')
            for t in types:
                if Pessimistic:
                    maskPessimistic = h5file[v][t]['Pessimistic'][...].squeeze() == True
                    seedsInterest = h5file[v][t]['randomSeed'][...].squeeze()[maskPessimistic]
                else:
                    seedsInterest = h5file[v][t]['randomSeed'][...].squeeze()
                seedsOriginal = originalDCOfile['doubleCompactObjects']['seed'][...].squeeze()
                #recover maskDCO for Bootstrap
                maskDCO       = np.in1d(seedsOriginal, seedsInterest)
                #for this type of this variation 
                for info in ['ObservedRate','IntrinsicRedshift0']:
                    if Pessimistic:
                        maskPessimistic = h5file[v][t]['Pessimistic'][...].squeeze() == True
                        ratePerSystem = h5file[v][t][info][...].squeeze()[maskPessimistic]
                    else:
                        ratePerSystem = h5file[v][t][info][...].squeeze()
                    centralValue  = np.sum(ratePerSystem)
                    lower, Upper  = bootstrap(ratePerSystem)
                    #fill in per three columns
                    results[counterRow][counterColumn+0] = centralValue
                    results[counterRow][counterColumn+1] = lower
                    results[counterRow][counterColumn+2] = Upper
                    counterColumn+=3
            counterRow += 1
        if Pessimistic:
            np.savetxt('RatesPessimistic.txt', results)
        else:
            np.savetxt('RatesOptimistic.txt', results)
        print 'Done'

# Creating the Tables 

In [7]:
o  = np.loadtxt('RatesOptimistic.txt')
p = np.loadtxt('RatesPessimistic.txt')

print o.shape

(18, 18)


In [8]:
starter     = ['Madau et al. & Ma et al. (2004) & 1 &',' & & 2 &',' & & 3 &',
               ' & Langer et al.  &1 &',\
               ' &  &2 &',\
               ' &  &3 &',\
               ' & Langer et al., offset  &1 &',\
               ' &  &2 &',\
               ' &  &3 &',\
               'Strolger et al. & Ma et al. (2004) & 1 &',\
               ' & &2 &',\
               ' & &3 &',\
               ' & Langer et al. &1 &',\
               ' &  &2 &',\
               ' &  &3 &',\
               ' & Langer et al., offset &1 &',\
               ' &  &2 &',\
               ' &  &3 &',\
              ]
print r'\begin{table*}'
print r'\begin{tabular}{|l |l | l| llllll}'
print r'\multicolumn{3}{|c|}{Variation MSSFR} & \multicolumn{6}{|c|}{ Rates}  \\'
print r'SFR & GSMF & MZ & BBH Obs &  BBH z=0  & BHNS Obs  & BHNS z=0 &  BNS Obs & BNS z=0 \\ \hline \hline'

print r' &&&&&&&&\\'
print r'\multicolumn{9}{|c|}{Pessimistic} \\'
print r' &&&&&&&&\\'

for nrV in range(len(h5file.keys())):
    string = starter[nrV]
    row    = p[nrV]
    row = np.round(row, 2)
    string+= r'$ %s_{%s}^{+%s} $ & $ %s_{%s}^{+%s} $ & $ %s_{%s}^{+%s} $ & $ %s_{%s}^{+%s} $ & $ %s_{%s}^{+%s}$ & $ %s_{%s}^{+%s} $ \\'\
    %(row[3],row[4], row[5],row[0],row[1],row[2],row[9],row[10],row[11],\
      row[6],row[7],row[8],row[15],row[16],row[17],row[12],row[13],row[14])
        
    if nrV in [2,5,11,14]:
        string += '\cline{2-9}'
    if nrV == 8:
        string += '\hline'
    print string

print r'\hline '
print r' &&&&&&&&\\'
print r'\multicolumn{9}{|c|}{Optimistic} \\'    
print r' &&&&&&&&\\'

for nrV in range(len(h5file.keys())):
    string = starter[nrV]
    row    = o[nrV]
    row = np.round(row, 2)
    string+= r'$ %s_{%s}^{+%s} $ & $ %s_{%s}^{+%s} $ & $ %s_{%s}^{+%s} $ & $ %s_{%s}^{+%s} $ & $ %s_{%s}^{+%s}$ & $ %s_{%s}^{+%s} $  \\'\
    %(row[3],row[4], row[5],row[0],row[1],row[2],row[9],row[10],row[11],\
      row[6],row[7],row[8],row[15],row[16],row[17],row[12],row[13],row[14])
        
    if nrV in [2,5,11,14]:
        string += '\cline{2-9}'
    if nrV == 8:
        string += '\hline'
    print string

print r'\end{tabular}'
print r'\end{table*}'


\begin{table*}
\begin{tabular}{|l |l | l| llllll}
\multicolumn{3}{|c|}{Variation MSSFR} & \multicolumn{6}{|c|}{ Rates}  \\
SFR & GSMF & MZ & BBH Obs &  BBH z=0  & BHNS Obs  & BHNS z=0 &  BNS Obs & BNS z=0 \\ \hline \hline
 &&&&&&&&\\
\multicolumn{9}{|c|}{Pessimistic} \\
 &&&&&&&&\\
Madau et al. & Ma et al. (2004) & 1 &$ 63.07_{-1.76}^{+1.86} $ & $ 18.43_{-0.4}^{+0.42} $ & $ 32.22_{-1.43}^{+1.7} $ & $ 0.4_{-0.02}^{+0.02} $ & $ 85.97_{-2.41}^{+2.47}$ & $ 0.11_{-0.0}^{+0.0} $ \\
 & & 2 &$ 158.56_{-2.48}^{+2.07} $ & $ 94.35_{-1.39}^{+1.37} $ & $ 40.73_{-1.13}^{+1.37} $ & $ 0.51_{-0.02}^{+0.02} $ & $ 90.87_{-2.15}^{+2.39}$ & $ 0.12_{-0.0}^{+0.0} $ \\
 & & 3 &$ 174.71_{-2.7}^{+2.28} $ & $ 113.92_{-1.22}^{+1.12} $ & $ 42.14_{-1.22}^{+1.43} $ & $ 0.52_{-0.02}^{+0.02} $ & $ 91.42_{-2.24}^{+2.37}$ & $ 0.13_{-0.0}^{+0.0} $ \\\cline{2-9}
 & Langer et al.  &1 &$ 448.84_{-5.24}^{+4.28} $ & $ 247.22_{-2.78}^{+2.63} $ & $ 95.47_{-2.12}^{+1.85} $ & $ 1.28_{-0.03}^{+0.03} $ & $ 144.57_{-2.3}^{+2.08}$ & 